In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [ ]:
# t = PromptTemplate.from_template("What is the capital of {country}?")
# # 이처럼 템플릿을 만들어서 입력할 수 있음 -> 템플릿을 만들어서 사용하면 유효성 검사를 할 수 있다는 장점이 있음
# # 또, 템플릿을 사용하는 이유중 하나는 promt template을 디스크에 저장하고 로드할 수 있기 때문
# # .from_template는 short cut

# # # short cut을 사용하지 않으면 이렇게 쓸 수 있음
# # t = PromptTemplate(
# #     template="What is the capital of {country}?",
# #     input_variables=["country"],
# # )

# t.format(country="France")

In [7]:
# 기본적으로 fewshot은 모델에게 예제를 주는 건데
# 더 나은 대답을 할 수 있도록 하는 예제들을 더 줄 수 있다는 거

# 구체적으로 대답하는 AI 모델이 필요하다고 할때, 어떻게 대답해야 하는지에 대한 예제를 주는것이 prompt를 사용해서 어떻게 대답해야 하는지 알려주는 것보다 좋음
# 즉, prompt에서 콤마써서 구분해줘, 소문자만 써야돼 등등 해서 넣는거보다 하나의 텍스트 예제를 입력해주는게 좋다는 말 -> 어차피 모델은 텍스트를 만들기 때문에
# FewShotPromptTemplate은 이런 예제를 FewShot 예제로 해주는거

# 예제들을 통해서 형식화(format)할 수 있음 -> 예제들이 DB에 있을수도 있기 때문에
# CS 챗봇을 만든다고 할때,
# 많은 고객들과의 대화 기록이나 다른 많은 기록들이 있을 거고, 모델이 어떻게 고객들에게 대응하는지 알려주고 싶을 거
# 대화기록 같은 것들을 DB에서 가져와서 FewShotPtomptTemplate을 사용하여 형식화하면 더 빠르게 Prompt를 만들 수 있을 것

In [8]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

In [10]:
# FewShotPromptTemplate 안썼을때
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. France is also known for its cuisine, including dishes such as croissants, baguettes, and coq au vin. The country has a diverse landscape, including the Alps in the east, the Mediterranean coast in the south, and the Atlantic coastline in the west. France is also a founding member of the European Union and a permanent member of the United Nations Security Council.

'France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. France is also known for its cuisine, including dishes such as croissants, baguettes, and coq au vin. The country has a diverse landscape, including the Alps in the east, the Mediterranean coast in the south, and the Atlantic coastline in the west. France is also a founding member of the European Union and a permanent member of the United Nations Security Council.'

In [20]:
# FewShotPromptTemplate 썼을때
# example 안에 있는 키값과 같은 걸로 넣어줘야 함
example_template = """
Human: {question}
AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    examples=examples,  # 예제들을 가져오고
    example_prompt=example_prompt,  # 예제들을 형식화하고
    suffix="Human : What do you know about {country}?",  # 형식화된 모든 예제 마지막에 나오는 내용 -> 한번 다 형식화가 끝나고 사용자의 질문은 무엇인지 나오게끔
    input_variables=["country"],  # 유효성 검사를 위해서 입력변수 지정
)

prompt.format(country="Germany")

'\nHuman: What do you know about France?\nAI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\nHuman: What do you know about Italy?\nAI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\n\nHuman: What do you know about Greece?\nAI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\n\nHuman : What do you know about Germany?'

In [17]:
chain = prompt | chat
chain.invoke(
    {
        "country": "Korea",
    }
)

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean won')